In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import uproot

import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))


In [ ]:
filename = "../small_data_files/failing_ncpi0_ntuple_file_with_splines.root"
filename = "/nevis/riverside/data/leehagaman/ngem/data_files/smaller_partial_nu_overlay_with_splines.root"
filename = "/nevis/riverside/data/leehagaman/ngem/data_files/nu_overlay_splines_50.root"

uproot.open(filename)["nuselection"]["NeutrinoSelectionFilter"].items()

In [ ]:
distances = uproot.open(filename)["wcpselection"]["T_PFeval"].arrays(["mcflux_dk2gen", "mcflux_gen2vtx"], library="np")

plt.figure()
plt.hist(distances["mcflux_dk2gen"], bins=100, histtype="step")
plt.xlabel("Distance from neutrino production to GENIE generation (m)")
plt.ylabel("Number of events")
plt.show()

plt.figure()
plt.hist(distances["mcflux_gen2vtx"], bins=100, histtype="step")
plt.xlabel("Distance from GENIE generation to GENIE interaction (m)")
plt.ylabel("Number of events")
plt.show()

plt.figure()
plt.hist(distances["mcflux_gen2vtx"] + distances["mcflux_dk2gen"], bins=100, histtype="step")
plt.xlabel("Distance from neutrino production to GENIE interaction (m)")
plt.ylabel("Number of events")
plt.show()


In [ ]:
uproot.open(filename)["nuselection"]["NeutrinoSelectionFilter"]["weights"].items()

In [ ]:
spline_varnames = uproot.open(filename)["spline_weights"].keys()

spline_weight_dic = {}
for varname in tqdm(spline_varnames):
    weight_vector = uproot.open(filename)["spline_weights"][varname].arrays(library="np")[varname]
    stacked_weights = np.stack(weight_vector)
    spline_weight_dic[varname] = stacked_weights

In [ ]:
print_weights = True

for varname, weights in spline_weight_dic.items():
    is_2d = weights.ndim == 2
    if print_weights:
        if not is_2d:
            print(varname, weights.shape)
            print("          ", [float(f"{w:.2e}") for w in weights[:5]])
        else:
            print(varname, weights.shape, "(rows are universes, columns are events)")
            for weight_event in weights[:5]:
                print("          ", [float(f"{w:.2e}") for w in weight_event])
    else:
        print(varname, weights.shape)


In [ ]:
wc_energies = uproot.open(filename)["wcpselection"]["T_KINEvars"].arrays(["kine_reco_Enu"], library="np")["kine_reco_Enu"]
wc_truth_nuScatType = uproot.open(filename)["wcpselection"]["T_PFeval"].arrays(["truth_nuScatType"], library="np")["truth_nuScatType"]
wc_truth_isCC = uproot.open(filename)["wcpselection"]["T_eval"].arrays(["truth_isCC"], library="np")["truth_isCC"]
wc_truth_nuPdg = uproot.open(filename)["wcpselection"]["T_eval"].arrays(["truth_nuPdg"], library="np")["truth_nuPdg"]
wc_truth_NprimPio = uproot.open(filename)["wcpselection"]["T_PFeval"].arrays(["truth_NprimPio"], library="np")["truth_NprimPio"]
wc_truth_NCDelta = uproot.open(filename)["wcpselection"]["T_PFeval"].arrays(["truth_NCDelta"], library="np")["truth_NCDelta"]

weight_cv = uproot.open(filename)["wcpselection"]["T_eval"].arrays(["weight_cv"], library="np")["weight_cv"]
weight_spline = uproot.open(filename)["wcpselection"]["T_eval"].arrays(["weight_spline"], library="np")["weight_spline"]

cv_weights = weight_cv * weight_spline
cv_weights[cv_weights < 0] = 1
cv_weights[cv_weights > 30] = 1
cv_weights[np.isnan(cv_weights)] = 1
cv_weights[np.isinf(cv_weights)] = 1

num_events = spline_weight_dic["run"].shape[0]
num_events

# GENIE

In [ ]:
plotted_vars = []

In [ ]:
for varname in spline_weight_dic.keys():
    if "rad" in varname.lower() or "delta" in varname.lower() or "norm" in varname.lower() or "gamma" in varname.lower():
        print(varname)

See https://arxiv.org/abs/2110.14028 for variable descriptions

In [ ]:
cc_indices = np.where(wc_truth_isCC == 1)[0]
nc_indices = np.where(wc_truth_isCC == 0)[0]
numu_indices = np.where(np.abs(wc_truth_nuPdg) == 14)[0]

qe_indices = np.where(wc_truth_nuScatType == 1)[0]
res_indices = np.where(wc_truth_nuScatType == 4)[0]
dis_indices = np.where(wc_truth_nuScatType == 3)[0]
mec_indices = np.where(wc_truth_nuScatType == 10)[0]
coh_indices = np.where(wc_truth_nuScatType == 5)[0]

has_pi0_indices = np.where(wc_truth_NprimPio > 0)[0]
has_deltarad_indices = np.where(wc_truth_NCDelta > 0)[0]

numuCC_indices = np.intersect1d(numu_indices, cc_indices)
numuCCQE_indices = np.intersect1d(np.intersect1d(numu_indices, cc_indices), qe_indices)
numuCCMEC_indices = np.intersect1d(np.intersect1d(numu_indices, cc_indices), mec_indices)
NCMEC_indices = np.intersect1d(nc_indices, mec_indices)
ncpi0_indices = np.intersect1d(nc_indices, has_pi0_indices)
cc_coh_indices = np.intersect1d(cc_indices, coh_indices)
nc_coh_indices = np.intersect1d(nc_indices, coh_indices)
numuCCRES_indices = np.intersect1d(np.intersect1d(numu_indices, cc_indices), res_indices)
NCRES_indices = np.intersect1d(nc_indices, res_indices)
nonRES_indices = np.where(wc_truth_nuScatType != 4)[0]
nonRES_NC_indices = np.intersect1d(nonRES_indices, nc_indices)
nonRES_numuCC_indices = np.intersect1d(np.intersect1d(numu_indices, cc_indices), nonRES_indices)


## CCQE

In [ ]:
for varname, weights in spline_weight_dic.items():
    if not varname in ["MaCCQE_UBGenie", "AxFFCCQEshape_UBGenie", "VecFFCCQEshape_UBGenie", "RPA_CCQE_UBGenie", "CoulombCCQE_UBGenie"]:
        continue
    num_unis = weights.shape[1]
    if varname in plotted_vars:
        print("double plotting!")
    plotted_vars.append(varname)

    fig, axs = plt.subplots(1, 2, figsize=(10, 5))

    curr_energies = [e for i, e in enumerate(wc_energies) if i in numuCCQE_indices]
    curr_cv_weights = [cv_weights[event_i] for event_i in range(num_events) if event_i in numuCCQE_indices]
    for uni_i in range(num_unis):
        event_weights = [weights[event_i][uni_i] for event_i in range(num_events) if event_i in numuCCQE_indices]
        event_weights = np.array(event_weights)
        num_negative_weights = np.sum(event_weights < 0)
        if num_negative_weights > 0:
            print(f"{varname} has {num_negative_weights}/{num_events} negative weights! First 10:", event_weights[event_weights < 0][:10])
        event_weights[event_weights > 30] = 1
        event_weights[event_weights < 0] = 1
        event_weights[np.isnan(event_weights)] = 1
        event_weights[np.isinf(event_weights)] = 1
        colors = plt.cm.gist_rainbow(np.linspace(0, 1, num_unis))
        axs[0].hist(curr_energies, weights=event_weights, bins=np.linspace(0, 2500, 26), histtype="step", color=colors[uni_i])
    axs[0].hist(curr_energies, weights=curr_cv_weights, bins=np.linspace(0, 2500, 26), histtype="step", color="black", lw=3, ls="--", zorder=-10)
    axs[0].set_title("True numuCC QE Events")
    axs[0].set_xlabel("WC kine_reco_Enu (MeV)")
    axs[0].set_ylabel("Summed weight")

    for uni_i in range(num_unis):
        event_weights = [weights[event_i][uni_i] for event_i in range(num_events)]
        event_weights = np.array(event_weights)
        num_negative_weights = np.sum(event_weights < 0)
        if num_negative_weights > 0:
            print(f"{varname} has {num_negative_weights}/{num_events} negative weights! First 10:", event_weights[event_weights < 0][:10])
        event_weights[event_weights > 30] = 1
        event_weights[event_weights < 0] = 1
        event_weights[np.isnan(event_weights)] = 1
        event_weights[np.isinf(event_weights)] = 1
        colors = plt.cm.gist_rainbow(np.linspace(0, 1, num_unis))
        axs[1].hist(wc_energies, weights=event_weights, bins=np.linspace(0, 2500, 26), histtype="step", color=colors[uni_i])
    axs[1].hist(wc_energies, weights=cv_weights, bins=np.linspace(0, 2500, 26), histtype="step", color="black", lw=3, ls="--", zorder=-10)
    axs[1].set_title("All Events")
    axs[1].set_xlabel("WC kine_reco_Enu (MeV)")
    axs[1].set_ylabel("Summed weight")

    plt.suptitle(f"{varname} with {num_unis} universes")
    plt.show()


## CC MEC

In [ ]:
for varname, weights in spline_weight_dic.items():
    if not varname in ["NormCCMEC_UBGenie", "XSecShape_CCMEC_UBGenie", "DecayAngMEC_UBGenie", "FracPN_CCMEC_UBGenie", "FracDelta_CCMEC_UBGenie"]:
        continue
    num_unis = weights.shape[1]
    if varname in plotted_vars:
        print("double plotting!")
    plotted_vars.append(varname)

    fig, axs = plt.subplots(1, 2, figsize=(10, 5))

    curr_energies = [e for i, e in enumerate(wc_energies) if i in numuCCMEC_indices]
    curr_cv_weights = [cv_weights[event_i] for event_i in range(num_events) if event_i in numuCCMEC_indices]
    for uni_i in range(num_unis):
        event_weights = [weights[event_i][uni_i] for event_i in range(num_events) if event_i in numuCCMEC_indices]
        event_weights = np.array(event_weights)
        num_negative_weights = np.sum(event_weights < 0)
        if num_negative_weights > 0:
            print(f"{varname} has {num_negative_weights}/{num_events} negative weights! First 10:", event_weights[event_weights < 0][:10])
        event_weights[event_weights > 30] = 1
        event_weights[event_weights < 0] = 1
        event_weights[np.isnan(event_weights)] = 1
        event_weights[np.isinf(event_weights)] = 1
        colors = plt.cm.gist_rainbow(np.linspace(0, 1, num_unis))
        energies = [e for i, e in enumerate(wc_energies) if i in numuCCMEC_indices]
        axs[0].hist(energies, weights=event_weights, bins=np.linspace(0, 2500, 26), histtype="step", color=colors[uni_i])
    axs[0].hist(curr_energies, weights=curr_cv_weights, bins=np.linspace(0, 2500, 26), histtype="step", color="black", lw=3, ls="--", zorder=-10)
    axs[0].set_title("True numuCC MEC Events")
    axs[0].set_xlabel("WC kine_reco_Enu (MeV)")
    axs[0].set_ylabel("Summed weight")

    for uni_i in range(num_unis):
        event_weights = [weights[event_i][uni_i] for event_i in range(num_events)]
        event_weights = np.array(event_weights)
        num_negative_weights = np.sum(event_weights < 0)
        if num_negative_weights > 0:
            print(f"{varname} has {num_negative_weights}/{num_events} negative weights! First 10:", event_weights[event_weights < 0][:10])
        event_weights[event_weights > 30] = 1
        event_weights[event_weights < 0] = 1
        event_weights[np.isnan(event_weights)] = 1
        event_weights[np.isinf(event_weights)] = 1
        colors = plt.cm.gist_rainbow(np.linspace(0, 1, num_unis))
        axs[1].hist(wc_energies, weights=event_weights, bins=np.linspace(0, 2500, 26), histtype="step", color=colors[uni_i])
    axs[1].hist(wc_energies, weights=cv_weights, bins=np.linspace(0, 2500, 26), histtype="step", color="black", lw=3, ls="--", zorder=-10)
    axs[1].set_title("All Events")
    axs[1].set_xlabel("WC kine_reco_Enu (MeV)")
    axs[1].set_ylabel("Summed weight")

    plt.suptitle(f"{varname} with {num_unis} universes")
    plt.show()


## NC MEC

In [ ]:
for varname, weights in spline_weight_dic.items():
    if not varname in ["NormNCMEC_UBGenie"]:
        continue
    num_unis = weights.shape[1]
    if varname in plotted_vars:
        print("double plotting!")
    plotted_vars.append(varname)

    fig, axs = plt.subplots(1, 2, figsize=(10, 5))

    curr_energies = [e for i, e in enumerate(wc_energies) if i in NCMEC_indices]
    curr_cv_weights = [cv_weights[event_i] for event_i in range(num_events) if event_i in NCMEC_indices]
    for uni_i in range(num_unis):
        event_weights = [weights[event_i][uni_i] for event_i in range(num_events) if event_i in NCMEC_indices]
        event_weights = np.array(event_weights)
        num_negative_weights = np.sum(event_weights < 0)
        if num_negative_weights > 0:
            print(f"{varname} has {num_negative_weights}/{num_events} negative weights! First 10:", event_weights[event_weights < 0][:10])
        event_weights[event_weights > 30] = 1
        event_weights[event_weights < 0] = 1
        event_weights[np.isnan(event_weights)] = 1
        event_weights[np.isinf(event_weights)] = 1
        colors = plt.cm.gist_rainbow(np.linspace(0, 1, num_unis))
        energies = [e for i, e in enumerate(wc_energies) if i in NCMEC_indices]
        axs[0].hist(energies, weights=event_weights, bins=np.linspace(0, 2500, 26), histtype="step", color=colors[uni_i])
    axs[0].hist(curr_energies, weights=curr_cv_weights, bins=np.linspace(0, 2500, 26), histtype="step", color="black", lw=3, ls="--", zorder=-10)
    axs[0].set_title("True NC MEC Events")
    axs[0].set_xlabel("WC kine_reco_Enu (MeV)")
    axs[0].set_ylabel("Summed weight")

    for uni_i in range(num_unis):
        event_weights = [weights[event_i][uni_i] for event_i in range(num_events)]
        event_weights = np.array(event_weights)
        num_negative_weights = np.sum(event_weights < 0)
        if num_negative_weights > 0:
            print(f"{varname} has {num_negative_weights}/{num_events} negative weights! First 10:", event_weights[event_weights < 0][:10])
        event_weights[event_weights > 30] = 1
        event_weights[event_weights < 0] = 1
        event_weights[np.isnan(event_weights)] = 1
        event_weights[np.isinf(event_weights)] = 1
        colors = plt.cm.gist_rainbow(np.linspace(0, 1, num_unis))
        axs[1].hist(wc_energies, weights=event_weights, bins=np.linspace(0, 2500, 26), histtype="step", color=colors[uni_i])
    axs[1].hist(wc_energies, weights=cv_weights, bins=np.linspace(0, 2500, 26), histtype="step", color="black", lw=3, ls="--", zorder=-10)
    axs[1].set_title("All Events")
    axs[1].set_xlabel("WC kine_reco_Enu (MeV)")
    axs[1].set_ylabel("Summed weight")

    plt.suptitle(f"{varname} with {num_unis} universes")
    plt.show()


## Delta Rad

In [ ]:
for varname, weights in spline_weight_dic.items():
    if not varname in ["RDecBR1gamma_UBGenie", "ThetaDelta2NRad_UBGenie"]:
        continue
    num_unis = weights.shape[1]
    if varname in plotted_vars:
        print("double plotting!")
    plotted_vars.append(varname)

    fig, axs = plt.subplots(1, 2, figsize=(10, 5))

    curr_energies = [e for i, e in enumerate(wc_energies) if i in has_deltarad_indices]
    curr_cv_weights = [cv_weights[event_i] for event_i in range(num_events) if event_i in has_deltarad_indices]
    for uni_i in range(num_unis):
        event_weights = [weights[event_i][uni_i] for event_i in range(num_events) if event_i in has_deltarad_indices]
        event_weights = np.array(event_weights)
        num_negative_weights = np.sum(event_weights < 0)
        if num_negative_weights > 0:
            print(f"{varname} has {num_negative_weights}/{num_events} negative weights! First 10:", event_weights[event_weights < 0][:10])
        event_weights[event_weights > 30] = 1
        event_weights[event_weights < 0] = 1
        event_weights[np.isnan(event_weights)] = 1
        event_weights[np.isinf(event_weights)] = 1
        colors = plt.cm.gist_rainbow(np.linspace(0, 1, num_unis))
        energies = [e for i, e in enumerate(wc_energies) if i in has_deltarad_indices]
        axs[0].hist(energies, weights=event_weights, bins=np.linspace(0, 2500, 26), histtype="step", color=colors[uni_i])
    axs[0].hist(curr_energies, weights=curr_cv_weights, bins=np.linspace(0, 2500, 26), histtype="step", color="black", lw=3, ls="--", zorder=-10)
    axs[0].set_title("True Delta Rad Events")
    axs[0].set_xlabel("WC kine_reco_Enu (MeV)")
    axs[0].set_ylabel("Summed weight")

    for uni_i in range(num_unis):
        event_weights = [weights[event_i][uni_i] for event_i in range(num_events)]
        event_weights = np.array(event_weights)
        num_negative_weights = np.sum(event_weights < 0)
        if num_negative_weights > 0:
            print(f"{varname} has {num_negative_weights}/{num_events} negative weights! First 10:", event_weights[event_weights < 0][:10])
        event_weights[event_weights > 30] = 1
        event_weights[event_weights < 0] = 1
        event_weights[np.isnan(event_weights)] = 1
        event_weights[np.isinf(event_weights)] = 1
        colors = plt.cm.gist_rainbow(np.linspace(0, 1, num_unis))
        axs[1].hist(wc_energies, weights=event_weights, bins=np.linspace(0, 2500, 26), histtype="step", color=colors[uni_i])
    axs[1].hist(wc_energies, weights=cv_weights, bins=np.linspace(0, 2500, 26), histtype="step", color="black", lw=3, ls="--", zorder=-10)
    axs[1].set_title("All Events")
    axs[1].set_xlabel("WC kine_reco_Enu (MeV)")
    axs[1].set_ylabel("Summed weight")

    plt.suptitle(f"{varname} with {num_unis} universes")
    plt.show()


## CC COH

In [ ]:
for varname, weights in spline_weight_dic.items():
    if not varname in ["NormCCCOH_UBGenie"]:
        continue
    num_unis = weights.shape[1]
    if varname in plotted_vars:
        print("double plotting!")
    plotted_vars.append(varname)

    fig, axs = plt.subplots(1, 2, figsize=(10, 5))

    curr_energies = [e for i, e in enumerate(wc_energies) if i in cc_coh_indices]
    curr_cv_weights = [cv_weights[event_i] for event_i in range(num_events) if event_i in cc_coh_indices]
    for uni_i in range(num_unis):
        event_weights = [weights[event_i][uni_i] for event_i in range(num_events) if event_i in cc_coh_indices]
        event_weights = np.array(event_weights)
        num_negative_weights = np.sum(event_weights < 0)
        if num_negative_weights > 0:
            print(f"{varname} has {num_negative_weights}/{num_events} negative weights! First 10:", event_weights[event_weights < 0][:10])
        event_weights[event_weights > 30] = 1
        event_weights[event_weights < 0] = 1
        event_weights[np.isnan(event_weights)] = 1
        event_weights[np.isinf(event_weights)] = 1
        colors = plt.cm.gist_rainbow(np.linspace(0, 1, num_unis))
        energies = [e for i, e in enumerate(wc_energies) if i in cc_coh_indices]
        axs[0].hist(energies, weights=event_weights, bins=np.linspace(0, 2500, 26), histtype="step", color=colors[uni_i])
    axs[0].hist(curr_energies, weights=curr_cv_weights, bins=np.linspace(0, 2500, 26), histtype="step", color="black", lw=3, ls="--", zorder=-10)
    axs[0].set_title("True CC COH Events")
    axs[0].set_xlabel("WC kine_reco_Enu (MeV)")
    axs[0].set_ylabel("Summed weight")

    for uni_i in range(num_unis):
        event_weights = [weights[event_i][uni_i] for event_i in range(num_events)]
        event_weights = np.array(event_weights)
        num_negative_weights = np.sum(event_weights < 0)
        if num_negative_weights > 0:
            print(f"{varname} has {num_negative_weights}/{num_events} negative weights! First 10:", event_weights[event_weights < 0][:10])
        event_weights[event_weights > 30] = 1
        event_weights[event_weights < 0] = 1
        event_weights[np.isnan(event_weights)] = 1
        event_weights[np.isinf(event_weights)] = 1
        colors = plt.cm.gist_rainbow(np.linspace(0, 1, num_unis))
        axs[1].hist(wc_energies, weights=event_weights, bins=np.linspace(0, 2500, 26), histtype="step", color=colors[uni_i])
    axs[1].hist(wc_energies, weights=cv_weights, bins=np.linspace(0, 2500, 26), histtype="step", color="black", lw=3, ls="--", zorder=-10)
    axs[1].set_title("All Events")
    axs[1].set_xlabel("WC kine_reco_Enu (MeV)")
    axs[1].set_ylabel("Summed weight")

    plt.suptitle(f"{varname} with {num_unis} universes")
    plt.show()


## NC COH

In [ ]:
for varname, weights in spline_weight_dic.items():
    if not varname in ["NormNCCOH_UBGenie"]:
        continue
    num_unis = weights.shape[1]
    if varname in plotted_vars:
        print("double plotting!", varname)
    plotted_vars.append(varname)

    fig, axs = plt.subplots(1, 2, figsize=(10, 5))

    curr_energies = [e for i, e in enumerate(wc_energies) if i in nc_coh_indices]
    curr_cv_weights = [cv_weights[event_i] for event_i in range(num_events) if event_i in nc_coh_indices]
    for uni_i in range(num_unis):
        event_weights = [weights[event_i][uni_i] for event_i in range(num_events) if event_i in nc_coh_indices]
        event_weights = np.array(event_weights)
        num_negative_weights = np.sum(event_weights < 0)
        if num_negative_weights > 0:
            print(f"{varname} has {num_negative_weights}/{num_events} negative weights! First 10:", event_weights[event_weights < 0][:10])
        event_weights[event_weights > 30] = 1
        event_weights[event_weights < 0] = 1
        event_weights[np.isnan(event_weights)] = 1
        event_weights[np.isinf(event_weights)] = 1
        colors = plt.cm.gist_rainbow(np.linspace(0, 1, num_unis))
        energies = [e for i, e in enumerate(wc_energies) if i in nc_coh_indices]
        axs[0].hist(energies, weights=event_weights, bins=np.linspace(0, 2500, 26), histtype="step", color=colors[uni_i])
    axs[0].hist(curr_energies, weights=curr_cv_weights, bins=np.linspace(0, 2500, 26), histtype="step", color="black", lw=3, ls="--", zorder=-10)
    axs[0].set_title("True NC COH Events")
    axs[0].set_xlabel("WC kine_reco_Enu (MeV)")
    axs[0].set_ylabel("Summed weight")
    axs[0].set_ylim(-10, 50)

    for uni_i in range(num_unis):
        event_weights = [weights[event_i][uni_i] for event_i in range(num_events)]
        event_weights = np.array(event_weights)
        num_negative_weights = np.sum(event_weights < 0)
        if num_negative_weights > 0:
            print(f"{varname} has {num_negative_weights}/{num_events} negative weights! First 10:", event_weights[event_weights < 0][:10])
        event_weights[event_weights > 30] = 1
        event_weights[event_weights < 0] = 1
        event_weights[np.isnan(event_weights)] = 1
        event_weights[np.isinf(event_weights)] = 1
        colors = plt.cm.gist_rainbow(np.linspace(0, 1, num_unis))
        axs[1].hist(wc_energies, weights=event_weights, bins=np.linspace(0, 2500, 26), histtype="step", color=colors[uni_i])
    axs[1].hist(wc_energies, weights=cv_weights, bins=np.linspace(0, 2500, 26), histtype="step", color="black", lw=3, ls="--", zorder=-10)
    axs[1].set_title("All Events")
    axs[1].set_xlabel("WC kine_reco_Enu (MeV)")
    axs[1].set_ylabel("Summed weight")

    plt.suptitle(f"{varname} with {num_unis} universes")
    plt.show()


## NC

In [ ]:
for varname, weights in spline_weight_dic.items():
    if not varname in ["MaNCEL_UBGenie", "EtaNCEL_UBGenie"]:
        continue
    num_unis = weights.shape[1]
    if varname in plotted_vars:
        print("double plotting!", varname)
    plotted_vars.append(varname)

    fig, axs = plt.subplots(1, 2, figsize=(10, 5))

    curr_energies = [e for i, e in enumerate(wc_energies) if i in nc_indices]
    curr_cv_weights = [cv_weights[event_i] for event_i in range(num_events) if event_i in nc_indices]
    for uni_i in range(num_unis):
        event_weights = [weights[event_i][uni_i] for event_i in range(num_events) if event_i in nc_indices]
        event_weights = np.array(event_weights)
        num_negative_weights = np.sum(event_weights < 0)
        if num_negative_weights > 0:
            print(f"{varname} has {num_negative_weights}/{num_events} negative weights! First 10:", event_weights[event_weights < 0][:10])
        event_weights[event_weights > 30] = 1
        event_weights[event_weights < 0] = 1
        event_weights[np.isnan(event_weights)] = 1
        event_weights[np.isinf(event_weights)] = 1
        colors = plt.cm.gist_rainbow(np.linspace(0, 1, num_unis))
        energies = [e for i, e in enumerate(wc_energies) if i in nc_indices]
        axs[0].hist(energies, weights=event_weights, bins=np.linspace(0, 2500, 26), histtype="step", color=colors[uni_i])
    axs[0].hist(curr_energies, weights=curr_cv_weights, bins=np.linspace(0, 2500, 26), histtype="step", color="black", lw=3, ls="--", zorder=-10)
    axs[0].set_title("True NC Events")
    axs[0].set_xlabel("WC kine_reco_Enu (MeV)")
    axs[0].set_ylabel("Summed weight")

    for uni_i in range(num_unis):
        event_weights = [weights[event_i][uni_i] for event_i in range(num_events)]
        event_weights = np.array(event_weights)
        num_negative_weights = np.sum(event_weights < 0)
        if num_negative_weights > 0:
            print(f"{varname} has {num_negative_weights}/{num_events} negative weights! First 10:", event_weights[event_weights < 0][:10])
        event_weights[event_weights > 30] = 1
        event_weights[event_weights < 0] = 1
        event_weights[np.isnan(event_weights)] = 1
        event_weights[np.isinf(event_weights)] = 1
        colors = plt.cm.gist_rainbow(np.linspace(0, 1, num_unis))
        axs[1].hist(wc_energies, weights=event_weights, bins=np.linspace(0, 2500, 26), histtype="step", color=colors[uni_i])
    axs[1].hist(wc_energies, weights=cv_weights, bins=np.linspace(0, 2500, 26), histtype="step", color="black", lw=3, ls="--", zorder=-10)
    axs[1].set_title("All Events")
    axs[1].set_xlabel("WC kine_reco_Enu (MeV)")
    axs[1].set_ylabel("Summed weight")

    plt.suptitle(f"{varname} with {num_unis} universes")
    plt.show()


## CC RES

In [ ]:
for varname, weights in spline_weight_dic.items():
    if not varname in ["MaCCRES_UBGenie", "MvCCRES_UBGenie", "RDecBR1eta_UBGenie", "Theta_Delta2Npi_UBGenie"]:
        continue
    num_unis = weights.shape[1]
    if varname in plotted_vars:
        print("double plotting!", varname)
    plotted_vars.append(varname)

    fig, axs = plt.subplots(1, 2, figsize=(10, 5))

    curr_energies = [e for i, e in enumerate(wc_energies) if i in numuCCRES_indices]
    curr_cv_weights = [cv_weights[event_i] for event_i in range(num_events) if event_i in numuCCRES_indices]
    for uni_i in range(num_unis):
        event_weights = [weights[event_i][uni_i] for event_i in range(num_events) if event_i in numuCCRES_indices]
        event_weights = np.array(event_weights)
        num_negative_weights = np.sum(event_weights < 0)
        if num_negative_weights > 0:
            print(f"{varname} has {num_negative_weights}/{num_events} negative weights! First 10:", event_weights[event_weights < 0][:10])
        event_weights[event_weights > 30] = 1
        event_weights[event_weights < 0] = 1
        event_weights[np.isnan(event_weights)] = 1
        event_weights[np.isinf(event_weights)] = 1
        colors = plt.cm.gist_rainbow(np.linspace(0, 1, num_unis))
        energies = [e for i, e in enumerate(wc_energies) if i in numuCCRES_indices]
        axs[0].hist(energies, weights=event_weights, bins=np.linspace(0, 2500, 26), histtype="step", color=colors[uni_i])
    axs[0].hist(curr_energies, weights=curr_cv_weights, bins=np.linspace(0, 2500, 26), histtype="step", color="black", lw=3, ls="--", zorder=-10)
    axs[0].set_title("True numuCC RES Events")
    axs[0].set_xlabel("WC kine_reco_Enu (MeV)")
    axs[0].set_ylabel("Summed weight")

    for uni_i in range(num_unis):
        event_weights = [weights[event_i][uni_i] for event_i in range(num_events)]
        event_weights = np.array(event_weights)
        num_negative_weights = np.sum(event_weights < 0)
        if num_negative_weights > 0:
            print(f"{varname} has {num_negative_weights}/{num_events} negative weights! First 10:", event_weights[event_weights < 0][:10])
        event_weights[event_weights > 30] = 1
        event_weights[event_weights < 0] = 1
        event_weights[np.isnan(event_weights)] = 1
        event_weights[np.isinf(event_weights)] = 1
        colors = plt.cm.gist_rainbow(np.linspace(0, 1, num_unis))
        axs[1].hist(wc_energies, weights=event_weights, bins=np.linspace(0, 2500, 26), histtype="step", color=colors[uni_i])
    axs[1].hist(wc_energies, weights=cv_weights, bins=np.linspace(0, 2500, 26), histtype="step", color="black", lw=3, ls="--", zorder=-10)
    axs[1].set_title("All Events")
    axs[1].set_xlabel("WC kine_reco_Enu (MeV)")
    axs[1].set_ylabel("Summed weight")

    plt.suptitle(f"{varname} with {num_unis} universes")
    plt.show()


## NC RES

In [ ]:
for varname, weights in spline_weight_dic.items():
    if not varname in ["MaNCRES_UBGenie", "MvNCRES_UBGenie"]:
        continue
    num_unis = weights.shape[1]
    if varname in plotted_vars:
        print("double plotting!", varname)
    plotted_vars.append(varname)

    fig, axs = plt.subplots(1, 2, figsize=(10, 5))

    curr_energies = [e for i, e in enumerate(wc_energies) if i in NCRES_indices]
    curr_cv_weights = [cv_weights[event_i] for event_i in range(num_events) if event_i in NCRES_indices]
    for uni_i in range(num_unis):
        event_weights = [weights[event_i][uni_i] for event_i in range(num_events) if event_i in NCRES_indices]
        event_weights = np.array(event_weights)
        num_negative_weights = np.sum(event_weights < 0)
        if num_negative_weights > 0:
            print(f"{varname} has {num_negative_weights}/{num_events} negative weights! First 10:", event_weights[event_weights < 0][:10])
        event_weights[event_weights > 30] = 1
        event_weights[event_weights < 0] = 1
        event_weights[np.isnan(event_weights)] = 1
        event_weights[np.isinf(event_weights)] = 1
        colors = plt.cm.gist_rainbow(np.linspace(0, 1, num_unis))
        energies = [e for i, e in enumerate(wc_energies) if i in NCRES_indices]
        axs[0].hist(energies, weights=event_weights, bins=np.linspace(0, 2500, 26), histtype="step", color=colors[uni_i])
    axs[0].hist(curr_energies, weights=curr_cv_weights, bins=np.linspace(0, 2500, 26), histtype="step", color="black", lw=3, ls="--", zorder=-10)
    axs[0].set_title("True NC RES Events")
    axs[0].set_xlabel("WC kine_reco_Enu (MeV)")
    axs[0].set_ylabel("Summed weight")

    for uni_i in range(num_unis):
        event_weights = [weights[event_i][uni_i] for event_i in range(num_events)]
        event_weights = np.array(event_weights)
        num_negative_weights = np.sum(event_weights < 0)
        if num_negative_weights > 0:
            print(f"{varname} has {num_negative_weights}/{num_events} negative weights! First 10:", event_weights[event_weights < 0][:10])
        event_weights[event_weights > 30] = 1
        event_weights[event_weights < 0] = 1
        event_weights[np.isnan(event_weights)] = 1
        event_weights[np.isinf(event_weights)] = 1
        colors = plt.cm.gist_rainbow(np.linspace(0, 1, num_unis))
        axs[1].hist(wc_energies, weights=event_weights, bins=np.linspace(0, 2500, 26), histtype="step", color=colors[uni_i])
    axs[1].hist(wc_energies, weights=cv_weights, bins=np.linspace(0, 2500, 26), histtype="step", color="black", lw=3, ls="--", zorder=-10)
    axs[1].set_title("All Events")
    axs[1].set_xlabel("WC kine_reco_Enu (MeV)")
    axs[1].set_ylabel("Summed weight")

    plt.suptitle(f"{varname} with {num_unis} universes")
    plt.show()


## AGKY Hadronization

In [ ]:
for varname, weights in spline_weight_dic.items():
    if not varname in ["AGKYxF1pi_UBGenie", "AGKYpT1pi_UBGenie"]:
        continue
    num_unis = weights.shape[1]
    if varname in plotted_vars:
        print("double plotting!", varname)
    plotted_vars.append(varname)

    fig, axs = plt.subplots(1, 2, figsize=(10, 5))

    curr_energies = [e for i, e in enumerate(wc_energies) if i in numuCCRES_indices]
    curr_cv_weights = [cv_weights[event_i] for event_i in range(num_events) if event_i in numuCCRES_indices]
    for uni_i in range(num_unis):
        event_weights = [weights[event_i][uni_i] for event_i in range(num_events) if event_i in numuCCRES_indices]
        event_weights = np.array(event_weights)
        num_negative_weights = np.sum(event_weights < 0)
        if num_negative_weights > 0:
            print(f"{varname} has {num_negative_weights}/{num_events} negative weights! First 10:", event_weights[event_weights < 0][:10])
        event_weights[event_weights > 30] = 1
        event_weights[event_weights < 0] = 1
        event_weights[np.isnan(event_weights)] = 1
        event_weights[np.isinf(event_weights)] = 1
        colors = plt.cm.gist_rainbow(np.linspace(0, 1, num_unis))
        energies = [e for i, e in enumerate(wc_energies) if i in numuCCRES_indices]
        axs[0].hist(energies, weights=event_weights, bins=np.linspace(0, 2500, 26), histtype="step", color=colors[uni_i])
    axs[0].hist(curr_energies, weights=curr_cv_weights, bins=np.linspace(0, 2500, 26), histtype="step", color="black", lw=3, ls="--", zorder=-10)
    axs[0].set_title("True numuCC RES Events")
    axs[0].set_xlabel("WC kine_reco_Enu (MeV)")
    axs[0].set_ylabel("Summed weight")

    for uni_i in range(num_unis):
        event_weights = [weights[event_i][uni_i] for event_i in range(num_events)]
        event_weights = np.array(event_weights)
        num_negative_weights = np.sum(event_weights < 0)
        if num_negative_weights > 0:
            print(f"{varname} has {num_negative_weights}/{num_events} negative weights! First 10:", event_weights[event_weights < 0][:10])
        event_weights[event_weights > 30] = 1
        event_weights[event_weights < 0] = 1
        event_weights[np.isnan(event_weights)] = 1
        event_weights[np.isinf(event_weights)] = 1
        colors = plt.cm.gist_rainbow(np.linspace(0, 1, num_unis))
        axs[1].hist(wc_energies, weights=event_weights, bins=np.linspace(0, 2500, 26), histtype="step", color=colors[uni_i])
    axs[1].hist(wc_energies, weights=cv_weights, bins=np.linspace(0, 2500, 26), histtype="step", color="black", lw=3, ls="--", zorder=-10)
    axs[1].set_title("All Events")
    axs[1].set_xlabel("WC kine_reco_Enu (MeV)")
    axs[1].set_ylabel("Summed weight")

    plt.suptitle(f"{varname} with {num_unis} universes")
    plt.show()


## Non-RES NC

In [ ]:
for varname, weights in spline_weight_dic.items():
    if not varname in [
        "NonRESBGvbarnNC1pi_UBGenie", 
        "NonRESBGvbarnNC2pi_UBGenie", 
        "NonRESBGvbarpNC1pi_UBGenie", 
        "NonRESBGvbarpNC2pi_UBGenie", 
        "NonRESBGvnNC1pi_UBGenie", 
        "NonRESBGvnNC2pi_UBGenie", 
        "NonRESBGvpNC1pi_UBGenie", 
        "NonRESBGvpNC2pi_UBGenie"]:
        continue
    num_unis = weights.shape[1]
    if varname in plotted_vars:
        print("double plotting!", varname)
    plotted_vars.append(varname)

    fig, axs = plt.subplots(1, 2, figsize=(10, 5))

    curr_energies = [e for i, e in enumerate(wc_energies) if i in nonRES_NC_indices]
    curr_cv_weights = [cv_weights[event_i] for event_i in range(num_events) if event_i in nonRES_NC_indices]
    for uni_i in range(num_unis):
        event_weights = [weights[event_i][uni_i] for event_i in range(num_events) if event_i in nonRES_NC_indices]
        event_weights = np.array(event_weights)
        num_negative_weights = np.sum(event_weights < 0)
        if num_negative_weights > 0:
            print(f"{varname} has {num_negative_weights}/{num_events} negative weights! First 10:", event_weights[event_weights < 0][:10])
        event_weights[event_weights > 30] = 1
        event_weights[event_weights < 0] = 1
        event_weights[np.isnan(event_weights)] = 1
        event_weights[np.isinf(event_weights)] = 1
        colors = plt.cm.gist_rainbow(np.linspace(0, 1, num_unis))
        energies = [e for i, e in enumerate(wc_energies) if i in nonRES_NC_indices]
        axs[0].hist(energies, weights=event_weights, bins=np.linspace(0, 2500, 26), histtype="step", color=colors[uni_i])
    axs[0].hist(curr_energies, weights=curr_cv_weights, bins=np.linspace(0, 2500, 26), histtype="step", color="black", lw=3, ls="--", zorder=-10)
    axs[0].set_title("Non-RES NC Events")
    axs[0].set_xlabel("WC kine_reco_Enu (MeV)")
    axs[0].set_ylabel("Summed weight")

    for uni_i in range(num_unis):
        event_weights = [weights[event_i][uni_i] for event_i in range(num_events)]
        event_weights = np.array(event_weights)
        num_negative_weights = np.sum(event_weights < 0)
        if num_negative_weights > 0:
            print(f"{varname} has {num_negative_weights}/{num_events} negative weights! First 10:", event_weights[event_weights < 0][:10])
        event_weights[event_weights > 30] = 1
        event_weights[event_weights < 0] = 1
        event_weights[np.isnan(event_weights)] = 1
        event_weights[np.isinf(event_weights)] = 1
        colors = plt.cm.gist_rainbow(np.linspace(0, 1, num_unis))
        axs[1].hist(wc_energies, weights=event_weights, bins=np.linspace(0, 2500, 26), histtype="step", color=colors[uni_i])
    axs[1].hist(wc_energies, weights=cv_weights, bins=np.linspace(0, 2500, 26), histtype="step", color="black", lw=3, ls="--", zorder=-10)
    axs[1].set_title("All Events")
    axs[1].set_xlabel("WC kine_reco_Enu (MeV)")
    axs[1].set_ylabel("Summed weight")

    plt.suptitle(f"{varname} with {num_unis} universes")
    plt.show()


## Non-RES CC

In [ ]:
for varname, weights in spline_weight_dic.items():
    if not varname in [
        "NonRESBGvbarnCC1pi_UBGenie", 
        "NonRESBGvbarnCC2pi_UBGenie", 
        "NonRESBGvbarpCC1pi_UBGenie", 
        "NonRESBGvbarpCC2pi_UBGenie", 
        "NonRESBGvnCC1pi_UBGenie", 
        "NonRESBGvnCC2pi_UBGenie", 
        "NonRESBGvpCC1pi_UBGenie", 
        "NonRESBGvpCC2pi_UBGenie"]:
        continue
    num_unis = weights.shape[1]
    if varname in plotted_vars:
        print("double plotting!", varname)
    plotted_vars.append(varname)

    fig, axs = plt.subplots(1, 2, figsize=(10, 5))

    curr_energies = [e for i, e in enumerate(wc_energies) if i in nonRES_numuCC_indices]
    curr_cv_weights = [cv_weights[event_i] for event_i in range(num_events) if event_i in nonRES_numuCC_indices]
    for uni_i in range(num_unis):
        event_weights = [weights[event_i][uni_i] for event_i in range(num_events) if event_i in nonRES_numuCC_indices]
        event_weights = np.array(event_weights)
        num_negative_weights = np.sum(event_weights < 0)
        if num_negative_weights > 0:
            print(f"{varname} has {num_negative_weights}/{num_events} negative weights! First 10:", event_weights[event_weights < 0][:10])
        event_weights[event_weights > 30] = 1
        event_weights[event_weights < 0] = 1
        event_weights[np.isnan(event_weights)] = 1
        event_weights[np.isinf(event_weights)] = 1
        colors = plt.cm.gist_rainbow(np.linspace(0, 1, num_unis))
        energies = [e for i, e in enumerate(wc_energies) if i in nonRES_numuCC_indices]
        axs[0].hist(energies, weights=event_weights, bins=np.linspace(0, 2500, 26), histtype="step", color=colors[uni_i])
    axs[0].hist(curr_energies, weights=curr_cv_weights, bins=np.linspace(0, 2500, 26), histtype="step", color="black", lw=3, ls="--", zorder=-10)
    axs[0].set_title("Non-RES numuCC Events")
    axs[0].set_xlabel("WC kine_reco_Enu (MeV)")
    axs[0].set_ylabel("Summed weight")

    for uni_i in range(num_unis):
        event_weights = [weights[event_i][uni_i] for event_i in range(num_events)]
        event_weights = np.array(event_weights)
        num_negative_weights = np.sum(event_weights < 0)
        if num_negative_weights > 0:
            print(f"{varname} has {num_negative_weights}/{num_events} negative weights! First 10:", event_weights[event_weights < 0][:10])
        event_weights[event_weights > 30] = 1
        event_weights[event_weights < 0] = 1
        event_weights[np.isnan(event_weights)] = 1
        event_weights[np.isinf(event_weights)] = 1
        colors = plt.cm.gist_rainbow(np.linspace(0, 1, num_unis))
        axs[1].hist(wc_energies, weights=event_weights, bins=np.linspace(0, 2500, 26), histtype="step", color=colors[uni_i])
    axs[1].hist(wc_energies, weights=cv_weights, bins=np.linspace(0, 2500, 26), histtype="step", color="black", lw=3, ls="--", zorder=-10)
    axs[1].set_title("All Events")
    axs[1].set_xlabel("WC kine_reco_Enu (MeV)")
    axs[1].set_ylabel("Summed weight")

    plt.suptitle(f"{varname} with {num_unis} universes")
    plt.show()


## Bodek-Yang

In [ ]:
for varname, weights in spline_weight_dic.items():
    if not varname in ["AhtBY_UBGenie", "BhtBY_UBGenie", "CV1uBY_UBGenie", "CV2uBY_UBGenie"]:
        continue
    num_unis = weights.shape[1]
    if varname in plotted_vars:
        print("double plotting!", varname)
    plotted_vars.append(varname)

    fig, axs = plt.subplots(1, 2, figsize=(10, 5))

    curr_energies = [e for i, e in enumerate(wc_energies) if i in numuCC_indices]
    curr_cv_weights = [cv_weights[event_i] for event_i in range(num_events) if event_i in numuCC_indices]
    for uni_i in range(num_unis):
        event_weights = [weights[event_i][uni_i] for event_i in range(num_events) if event_i in numuCC_indices]
        event_weights = np.array(event_weights)
        num_negative_weights = np.sum(event_weights < 0)
        if num_negative_weights > 0:
            print(f"{varname} has {num_negative_weights}/{num_events} negative weights! First 10:", event_weights[event_weights < 0][:10])
        event_weights[event_weights > 30] = 1
        event_weights[event_weights < 0] = 1
        event_weights[np.isnan(event_weights)] = 1
        event_weights[np.isinf(event_weights)] = 1
        colors = plt.cm.gist_rainbow(np.linspace(0, 1, num_unis))
        energies = [e for i, e in enumerate(wc_energies) if i in numuCC_indices]
        axs[0].hist(energies, weights=event_weights, bins=np.linspace(0, 2500, 26), histtype="step", color=colors[uni_i])
    axs[0].hist(curr_energies, weights=curr_cv_weights, bins=np.linspace(0, 2500, 26), histtype="step", color="black", lw=3, ls="--", zorder=-10)
    axs[0].set_title("numuCC Events")
    axs[0].set_xlabel("WC kine_reco_Enu (MeV)")
    axs[0].set_ylabel("Summed weight")

    for uni_i in range(num_unis):
        event_weights = [weights[event_i][uni_i] for event_i in range(num_events)]
        event_weights = np.array(event_weights)
        num_negative_weights = np.sum(event_weights < 0)
        if num_negative_weights > 0:
            print(f"{varname} has {num_negative_weights}/{num_events} negative weights! First 10:", event_weights[event_weights < 0][:10])
        event_weights[event_weights > 30] = 1
        event_weights[event_weights < 0] = 1
        event_weights[np.isnan(event_weights)] = 1
        event_weights[np.isinf(event_weights)] = 1
        colors = plt.cm.gist_rainbow(np.linspace(0, 1, num_unis))
        axs[1].hist(wc_energies, weights=event_weights, bins=np.linspace(0, 2500, 26), histtype="step", color=colors[uni_i])
    axs[1].hist(wc_energies, weights=cv_weights, bins=np.linspace(0, 2500, 26), histtype="step", color="black", lw=3, ls="--", zorder=-10)
    axs[1].set_title("All Events")
    axs[1].set_xlabel("WC kine_reco_Enu (MeV)")
    axs[1].set_ylabel("Summed weight")

    plt.suptitle(f"{varname} with {num_unis} universes")
    plt.show()


## FSI

In [ ]:
for varname, weights in spline_weight_dic.items():
    if not varname in ["MFP_pi_UBGenie", "MFP_N_UBGenie", 
                        "FrCEx_pi_UBGenie", "FrInel_pi_UBGenie", "FrAbs_pi_UBGenie", 
                        "FrCEx_N_UBGenie", "FrInel_N_UBGenie", "FrAbs_N_UBGenie"]:
        continue
    num_unis = weights.shape[1]
    if varname in plotted_vars:
        print("double plotting!", varname)
    plotted_vars.append(varname)

    fig, axs = plt.subplots(1, 2, figsize=(10, 5))

    curr_energies = [e for i, e in enumerate(wc_energies) if i in numuCC_indices]
    curr_cv_weights = [cv_weights[event_i] for event_i in range(num_events) if event_i in numuCC_indices]
    for uni_i in range(num_unis):
        event_weights = [weights[event_i][uni_i] for event_i in range(num_events) if event_i in numuCC_indices]
        event_weights = np.array(event_weights)
        num_negative_weights = np.sum(event_weights < 0)
        if num_negative_weights > 0:
            print(f"{varname} has {num_negative_weights}/{num_events} negative weights! First 10:", event_weights[event_weights < 0][:10])
        event_weights[event_weights > 30] = 1
        event_weights[event_weights < 0] = 1
        event_weights[np.isnan(event_weights)] = 1
        event_weights[np.isinf(event_weights)] = 1
        colors = plt.cm.gist_rainbow(np.linspace(0, 1, num_unis))
        energies = [e for i, e in enumerate(wc_energies) if i in numuCC_indices]
        axs[0].hist(energies, weights=event_weights, bins=np.linspace(0, 2500, 26), histtype="step", color=colors[uni_i])
    axs[0].hist(curr_energies, weights=curr_cv_weights, bins=np.linspace(0, 2500, 26), histtype="step", color="black", lw=3, ls="--", zorder=-10)
    axs[0].set_title("numuCC Events")
    axs[0].set_xlabel("WC kine_reco_Enu (MeV)")
    axs[0].set_ylabel("Summed weight")

    for uni_i in range(num_unis):
        event_weights = [weights[event_i][uni_i] for event_i in range(num_events)]
        event_weights = np.array(event_weights)
        num_negative_weights = np.sum(event_weights < 0)
        if num_negative_weights > 0:
            print(f"{varname} has {num_negative_weights}/{num_events} negative weights! First 10:", event_weights[event_weights < 0][:10])
        event_weights[event_weights > 30] = 1
        event_weights[event_weights < 0] = 1
        event_weights[np.isnan(event_weights)] = 1
        event_weights[np.isinf(event_weights)] = 1
        colors = plt.cm.gist_rainbow(np.linspace(0, 1, num_unis))
        axs[1].hist(wc_energies, weights=event_weights, bins=np.linspace(0, 2500, 26), histtype="step", color=colors[uni_i])
    axs[1].hist(wc_energies, weights=cv_weights, bins=np.linspace(0, 2500, 26), histtype="step", color="black", lw=3, ls="--", zorder=-10)
    axs[1].set_title("All Events")
    axs[1].set_xlabel("WC kine_reco_Enu (MeV)")
    axs[1].set_ylabel("Summed weight")

    plt.suptitle(f"{varname} with {num_unis} universes")
    plt.show()


## Second Class Currents

In [ ]:
for varname, weights in spline_weight_dic.items():
    if not varname in ["xsr_scc_Fa3_SCC", "xsr_scc_Fv3_SCC"]:
        continue
    num_unis = weights.shape[1]
    if varname in plotted_vars:
        print("double plotting!", varname)
    plotted_vars.append(varname)

    fig, axs = plt.subplots(1, 2, figsize=(10, 5))

    curr_energies = [e for i, e in enumerate(wc_energies) if i in numuCC_indices]
    curr_cv_weights = [cv_weights[event_i] for event_i in range(num_events) if event_i in numuCC_indices]
    for uni_i in range(num_unis):
        event_weights = [weights[event_i][uni_i] for event_i in range(num_events) if event_i in numuCC_indices]
        event_weights = np.array(event_weights)
        num_negative_weights = np.sum(event_weights < 0)
        if num_negative_weights > 0:
            print(f"{varname} has {num_negative_weights}/{num_events} negative weights! First 10:", event_weights[event_weights < 0][:10])
        event_weights[event_weights > 30] = 1
        event_weights[event_weights < 0] = 1
        event_weights[np.isnan(event_weights)] = 1
        event_weights[np.isinf(event_weights)] = 1
        colors = plt.cm.gist_rainbow(np.linspace(0, 1, num_unis))
        energies = [e for i, e in enumerate(wc_energies) if i in numuCC_indices]
        axs[0].hist(energies, weights=event_weights, bins=np.linspace(0, 2500, 26), histtype="step", color=colors[uni_i])
    axs[0].hist(curr_energies, weights=curr_cv_weights, bins=np.linspace(0, 2500, 26), histtype="step", color="black", lw=3, ls="--", zorder=-10)
    axs[0].set_title("numuCC Events")
    axs[0].set_xlabel("WC kine_reco_Enu (MeV)")
    axs[0].set_ylabel("Summed weight")

    for uni_i in range(num_unis):
        event_weights = [weights[event_i][uni_i] for event_i in range(num_events)]
        event_weights = np.array(event_weights)
        num_negative_weights = np.sum(event_weights < 0)
        if num_negative_weights > 0:
            print(f"{varname} has {num_negative_weights}/{num_events} negative weights! First 10:", event_weights[event_weights < 0][:10])
        event_weights[event_weights > 30] = 1
        event_weights[event_weights < 0] = 1
        event_weights[np.isnan(event_weights)] = 1
        event_weights[np.isinf(event_weights)] = 1
        colors = plt.cm.gist_rainbow(np.linspace(0, 1, num_unis))
        axs[1].hist(wc_energies, weights=event_weights, bins=np.linspace(0, 2500, 26), histtype="step", color=colors[uni_i])
    axs[1].hist(wc_energies, weights=cv_weights, bins=np.linspace(0, 2500, 26), histtype="step", color="black", lw=3, ls="--", zorder=-10)
    axs[1].set_title("All Events")
    axs[1].set_xlabel("WC kine_reco_Enu (MeV)")
    axs[1].set_ylabel("Summed weight")

    plt.suptitle(f"{varname} with {num_unis} universes")
    plt.show()


# Flux

## Horn

In [ ]:
for varname, weights in spline_weight_dic.items():
    if not varname in ["horncurrent_FluxUnisim", "expskin_FluxUnisim"]:
        continue
    num_unis = weights.shape[1]
    if varname in plotted_vars:
        print("double plotting!", varname)
    plotted_vars.append(varname)

    fig, axs = plt.subplots(1, 2, figsize=(10, 5))

    for uni_i in range(num_unis):
        event_weights = [weights[event_i][uni_i] for event_i in range(num_events) if event_i in numuCC_indices]
        event_weights = np.array(event_weights)
        num_negative_weights = np.sum(event_weights < 0)
        if num_negative_weights > 0:
            print(f"{varname} has {num_negative_weights}/{num_events} negative weights! First 10:", event_weights[event_weights < 0][:10])
        event_weights[event_weights > 30] = 1
        event_weights[event_weights < 0] = 1
        event_weights[np.isnan(event_weights)] = 1
        event_weights[np.isinf(event_weights)] = 1
        colors = plt.cm.gist_rainbow(np.linspace(0, 1, num_unis))
        energies = [e for i, e in enumerate(wc_energies) if i in numuCC_indices]
        axs[0].hist(energies, weights=event_weights, bins=np.linspace(0, 2500, 26), histtype="step", color=colors[uni_i])
    axs[0].set_title("numuCC Events")
    axs[0].set_xlabel("WC kine_reco_Enu (MeV)")
    axs[0].set_ylabel("Summed weight")

    for uni_i in range(num_unis):
        event_weights = [weights[event_i][uni_i] for event_i in range(num_events)]
        event_weights = np.array(event_weights)
        num_negative_weights = np.sum(event_weights < 0)
        if num_negative_weights > 0:
            print(f"{varname} has {num_negative_weights}/{num_events} negative weights! First 10:", event_weights[event_weights < 0][:10])
        event_weights[event_weights > 30] = 1
        event_weights[event_weights < 0] = 1
        event_weights[np.isnan(event_weights)] = 1
        event_weights[np.isinf(event_weights)] = 1
        colors = plt.cm.gist_rainbow(np.linspace(0, 1, num_unis))
        axs[1].hist(wc_energies, weights=event_weights, bins=np.linspace(0, 2500, 26), histtype="step", color=colors[uni_i])
    axs[1].set_title("All Events")
    axs[1].set_xlabel("WC kine_reco_Enu (MeV)")
    axs[1].set_ylabel("Summed weight")

    plt.suptitle(f"{varname} with {num_unis} universes")
    plt.show()


## Pion Production Cross Section From Covariance Matrix

In [ ]:
for varname, weights in spline_weight_dic.items():
    if not varname in ["piplus_PrimaryHadronSWCentralSplineVariation", "piminus_PrimaryHadronSWCentralSplineVariation"]:
        continue
    num_unis = weights.shape[1]
    if varname in plotted_vars:
        print("double plotting!", varname)
    plotted_vars.append(varname)

    fig, axs = plt.subplots(1, 2, figsize=(10, 5))

    for uni_i in range(20):
        event_weights = [weights[event_i][uni_i] for event_i in range(num_events) if event_i in numuCC_indices]
        event_weights = np.array(event_weights)
        num_negative_weights = np.sum(event_weights < 0)
        if num_negative_weights > 0:
            print(f"{varname} has {num_negative_weights}/{num_events} negative weights! First 10:", event_weights[event_weights < 0][:10])
        event_weights[event_weights > 30] = 1
        event_weights[event_weights < 0] = 1
        event_weights[np.isnan(event_weights)] = 1
        event_weights[np.isinf(event_weights)] = 1
        colors = plt.cm.gist_rainbow(np.linspace(0, 1, num_unis))
        energies = [e for i, e in enumerate(wc_energies) if i in numuCC_indices]
        axs[0].hist(energies, weights=event_weights, bins=np.linspace(0, 2500, 26), histtype="step", color=colors[uni_i])
    axs[0].set_title("numuCC Events")
    axs[0].set_xlabel("WC kine_reco_Enu (MeV)")
    axs[0].set_ylabel("Summed weight")

    for uni_i in range(20):
        event_weights = [weights[event_i][uni_i] for event_i in range(num_events)]
        event_weights = np.array(event_weights)
        num_negative_weights = np.sum(event_weights < 0)
        if num_negative_weights > 0:
            print(f"{varname} has {num_negative_weights}/{num_events} negative weights! First 10:", event_weights[event_weights < 0][:10])
        event_weights[event_weights > 30] = 1
        event_weights[event_weights < 0] = 1
        event_weights[np.isnan(event_weights)] = 1
        event_weights[np.isinf(event_weights)] = 1
        colors = plt.cm.gist_rainbow(np.linspace(0, 1, num_unis))
        axs[1].hist(wc_energies, weights=event_weights, bins=np.linspace(0, 2500, 26), histtype="step", color=colors[uni_i])
    axs[1].set_title("All Events")
    axs[1].set_xlabel("WC kine_reco_Enu (MeV)")
    axs[1].set_ylabel("Summed weight")

    plt.suptitle(f"{varname} with {num_unis} universes")
    plt.show()


## Kaons

In [ ]:
for varname, weights in spline_weight_dic.items():
    if not varname in ["kminus_PrimaryHadronNormalization", "kplus_PrimaryHadronFeynmanScaling", "kzero_PrimaryHadronSanfordWang"]:
        continue
    num_unis = weights.shape[1]
    if varname in plotted_vars:
        print("double plotting!", varname)
    plotted_vars.append(varname)

    fig, axs = plt.subplots(1, 2, figsize=(10, 5))

    for uni_i in range(num_unis):
        event_weights = [weights[event_i][uni_i] for event_i in range(num_events) if event_i in numuCC_indices]
        event_weights = np.array(event_weights)
        num_negative_weights = np.sum(event_weights < 0)
        if num_negative_weights > 0:
            print(f"{varname} has {num_negative_weights}/{num_events} negative weights! First 10:", event_weights[event_weights < 0][:10])
        event_weights[event_weights > 30] = 1
        event_weights[event_weights < 0] = 1
        event_weights[np.isnan(event_weights)] = 1
        event_weights[np.isinf(event_weights)] = 1
        colors = plt.cm.gist_rainbow(np.linspace(0, 1, num_unis))
        energies = [e for i, e in enumerate(wc_energies) if i in numuCC_indices]
        axs[0].hist(energies, weights=event_weights, bins=np.linspace(0, 2500, 26), histtype="step", color=colors[uni_i])
    axs[0].set_title("numuCC Events")
    axs[0].set_xlabel("WC kine_reco_Enu (MeV)")
    axs[0].set_ylabel("Summed weight")

    for uni_i in range(num_unis):
        event_weights = [weights[event_i][uni_i] for event_i in range(num_events)]
        event_weights = np.array(event_weights)
        num_negative_weights = np.sum(event_weights < 0)
        if num_negative_weights > 0:
            print(f"{varname} has {num_negative_weights}/{num_events} negative weights! First 10:", event_weights[event_weights < 0][:10])
        event_weights[event_weights > 30] = 1
        event_weights[event_weights < 0] = 1
        event_weights[np.isnan(event_weights)] = 1
        event_weights[np.isinf(event_weights)] = 1
        colors = plt.cm.gist_rainbow(np.linspace(0, 1, num_unis))
        axs[1].hist(wc_energies, weights=event_weights, bins=np.linspace(0, 2500, 26), histtype="step", color=colors[uni_i])
    axs[1].set_title("All Events")
    axs[1].set_xlabel("WC kine_reco_Enu (MeV)")
    axs[1].set_ylabel("Summed weight")

    plt.suptitle(f"{varname} with {num_unis} universes")
    plt.show()


## Nucleon XS

In [ ]:
for varname, weights in spline_weight_dic.items():
    if not varname in ["nucleoninexsec_FluxUnisim", "nucleonqexsec_FluxUnisim", "nucleontotxsec_FluxUnisim"]:
        continue
    num_unis = weights.shape[1]
    if varname in plotted_vars:
        print("double plotting!", varname)
    plotted_vars.append(varname)

    fig, axs = plt.subplots(1, 2, figsize=(10, 5))

    for uni_i in range(num_unis):
        event_weights = [weights[event_i][uni_i] for event_i in range(num_events) if event_i in numuCC_indices]
        event_weights = np.array(event_weights)
        num_negative_weights = np.sum(event_weights < 0)
        if num_negative_weights > 0:
            print(f"{varname} has {num_negative_weights}/{num_events} negative weights! First 10:", event_weights[event_weights < 0][:10])
        event_weights[event_weights > 30] = 1
        event_weights[event_weights < 0] = 1
        event_weights[np.isnan(event_weights)] = 1
        event_weights[np.isinf(event_weights)] = 1
        colors = plt.cm.gist_rainbow(np.linspace(0, 1, num_unis))
        energies = [e for i, e in enumerate(wc_energies) if i in numuCC_indices]
        axs[0].hist(energies, weights=event_weights, bins=np.linspace(0, 2500, 26), histtype="step", color=colors[uni_i])
    axs[0].set_title("numuCC Events")
    axs[0].set_xlabel("WC kine_reco_Enu (MeV)")
    axs[0].set_ylabel("Summed weight")

    for uni_i in range(num_unis):
        event_weights = [weights[event_i][uni_i] for event_i in range(num_events)]
        event_weights = np.array(event_weights)
        num_negative_weights = np.sum(event_weights < 0)
        if num_negative_weights > 0:
            print(f"{varname} has {num_negative_weights}/{num_events} negative weights! First 10:", event_weights[event_weights < 0][:10])
        event_weights[event_weights > 30] = 1
        event_weights[event_weights < 0] = 1
        event_weights[np.isnan(event_weights)] = 1
        event_weights[np.isinf(event_weights)] = 1
        colors = plt.cm.gist_rainbow(np.linspace(0, 1, num_unis))
        axs[1].hist(wc_energies, weights=event_weights, bins=np.linspace(0, 2500, 26), histtype="step", color=colors[uni_i])
    axs[1].set_title("All Events")
    axs[1].set_xlabel("WC kine_reco_Enu (MeV)")
    axs[1].set_ylabel("Summed weight")

    plt.suptitle(f"{varname} with {num_unis} universes")
    plt.show()


## Pion XS

In [ ]:
for varname, weights in spline_weight_dic.items():
    if not varname in ["pioninexsec_FluxUnisim", "pionqexsec_FluxUnisim", "piontotxsec_FluxUnisim"]:
        continue
    num_unis = weights.shape[1]
    if varname in plotted_vars:
        print("double plotting!", varname)
    plotted_vars.append(varname)

    fig, axs = plt.subplots(1, 2, figsize=(10, 5))

    for uni_i in range(num_unis):
        event_weights = [weights[event_i][uni_i] for event_i in range(num_events) if event_i in numuCC_indices]
        event_weights = np.array(event_weights)
        num_negative_weights = np.sum(event_weights < 0)
        if num_negative_weights > 0:
            print(f"{varname} has {num_negative_weights}/{num_events} negative weights! First 10:", event_weights[event_weights < 0][:10])
        event_weights[event_weights > 30] = 1
        event_weights[event_weights < 0] = 1
        event_weights[np.isnan(event_weights)] = 1
        event_weights[np.isinf(event_weights)] = 1
        colors = plt.cm.gist_rainbow(np.linspace(0, 1, num_unis))
        energies = [e for i, e in enumerate(wc_energies) if i in numuCC_indices]
        axs[0].hist(energies, weights=event_weights, bins=np.linspace(0, 2500, 26), histtype="step", color=colors[uni_i])
    axs[0].set_title("numuCC Events")
    axs[0].set_xlabel("WC kine_reco_Enu (MeV)")
    axs[0].set_ylabel("Summed weight")

    for uni_i in range(num_unis):
        event_weights = [weights[event_i][uni_i] for event_i in range(num_events)]
        event_weights = np.array(event_weights)
        num_negative_weights = np.sum(event_weights < 0)
        if num_negative_weights > 0:
            print(f"{varname} has {num_negative_weights}/{num_events} negative weights! First 10:", event_weights[event_weights < 0][:10])
        event_weights[event_weights > 30] = 1
        event_weights[event_weights < 0] = 1
        event_weights[np.isnan(event_weights)] = 1
        event_weights[np.isinf(event_weights)] = 1
        colors = plt.cm.gist_rainbow(np.linspace(0, 1, num_unis))
        axs[1].hist(wc_energies, weights=event_weights, bins=np.linspace(0, 2500, 26), histtype="step", color=colors[uni_i])
    axs[1].set_title("All Events")
    axs[1].set_xlabel("WC kine_reco_Enu (MeV)")
    axs[1].set_ylabel("Summed weight")

    plt.suptitle(f"{varname} with {num_unis} universes")
    plt.show()


In [ ]:
for varname in spline_weight_dic.keys():
    if varname not in plotted_vars:
        print(varname)
